### to concat files from folders 

In [ ]:
import os
import pandas as pd

def read_excel_files(folder_path):
    files = os.listdir(folder_path)
    dataframes = []
    for file in files:
        if file.endswith('.xlsx'):
            file_path = os.path.join(folder_path, file)
            df = pd.read_excel(file_path)
            df['DATE'] = file.split('.')[0]
            dataframes.append(df)
    return pd.concat(dataframes, ignore_index=True)

folders = ['07','08','09','10','11','12']
dfs = []

base_path = r'W:\CO\ДЕПАРТАМЕНТ ФИНАНСОВЫХ РИСКОВ И ПОРТФЕЛЬНОГО АНАЛИЗА\Управление финансовых рисков\Алтынай\ЦБ\2022'

for folder in folders:
    folder_path = os.path.join(base_path, folder)
    if os.path.exists(folder_path):
        df = read_excel_files(folder_path)
        dfs.append(df)

concatenated_df = pd.concat(dfs, ignore_index=True)



### to concat excel sheets from excel file 

In [ ]:

# os.chdir(r'W:\CO\ДЕПАРТАМЕНТ ФИНАНСОВЫХ РИСКОВ И ПОРТФЕЛЬНОГО АНАЛИЗА\Управление финансовых рисков\Алтынай\ЦБ\2023')
# excel_files = [  'Портфель ЦБ 10 23.xlsx',
#  'Портфель ЦБ 11 23.xlsx',
#  'Портфель ЦБ 12 23.xlsx']

# dfs = []

# # Read all sheets from each file and append to the list
# for file in excel_files:
#     xl = pd.ExcelFile(file)
#     for sheet_name in xl.sheet_names:
#         df = xl.parse(sheet_name)
#         df['DATE'] = sheet_name  # Add a 'DATE' column with the sheet name
#         dfs.append(df)

# concatenated_df_new = pd.concat(dfs, ignore_index=True)

In [ ]:
import pandas as pd
import cx_Oracle

# Connection details
dsn = cx_Oracle.makedsn('10.23.13.62', '1521', 'OPERRISKDB')
connection = cx_Oracle.connect('OPERRISK_DS', 'Qwerty2020##', dsn)
cursor = connection.cursor()

# Sample DataFrame

# Table name to be created
table_name = 'NELLI_CB_NEW_VERSION'

# Dictionary mapping DataFrame dtypes to Oracle data types
data_types_mapping = {
    'int64': 'NUMBER',
    'float64': 'NUMBER',
    'object': 'VARCHAR2(255)',
    'datetime64[ns]': 'DATE'
}

# Generate the CREATE TABLE query
create_table_query = f"CREATE TABLE {table_name} ("

for column_name, dtype in new_data.dtypes.items():
    oracle_data_type = data_types_mapping.get(str(dtype), 'VARCHAR2(255)')
    
    # Check for valid column names
    if not column_name.strip():
        raise ValueError("Empty column name found in DataFrame")
    
    # Use double quotes for the DATE column name
    if column_name == 'DATE':
        column_name = '"DATE"'
    
    create_table_query += f"{column_name} {oracle_data_type}, "

create_table_query = create_table_query.rstrip(', ') + ")"

try:
    # Execute the CREATE TABLE query
    print(f"Executing query: {create_table_query}")
    cursor.execute(create_table_query)
    # Commit the changes
    connection.commit()
    print(f"Table '{table_name}' created successfully.")
except cx_Oracle.DatabaseError as e:
    error, = e.args
    print(f"Error {error.code}: {error.message}")
    print(f"Problematic query: {create_table_query}")

# Close the cursor and connection
cursor.close()
connection.close()

In [ ]:
import pandas as pd
import cx_Oracle
import datetime

# Connection details
os.chdir(r"W:\CO\ДЕПАРТАМЕНТ ФИНАНСОВЫХ РИСКОВ И ПОРТФЕЛЬНОГО АНАЛИЗА\Управление финансовых рисков\Нелли")

dsn = cx_Oracle.makedsn('10.23.13.62', '1521', 'OPERRISKDB')
connection = cx_Oracle.connect('OPERRISK_DS', 'Qwerty2020##', dsn)
cursor = connection.cursor()

# Table name to insert data into
table_name = 'NELLI_CB_NEW_VERSION'

try:
    for _, row in new_data.iterrows():
        placeholders = ', '.join([':' + str(i+1) for i in range(len(row))])
        values = []
        for val in row:
            if pd.isnull(val):
                values.append(None)
            elif isinstance(val, pd.Timestamp):  # Check if the value is a pandas Timestamp (datetime)
                values.append(val.to_pydatetime())
            else:
                values.append(str(val))  # Convert to string for non-datetime columns
        query = f"INSERT INTO {table_name} VALUES ({placeholders})"
        cursor.execute(query, values)

    # Commit the changes
    connection.commit()
    print("Data inserted successfully.")
except cx_Oracle.DatabaseError as e:
    error, = e.args
    print(f"Error {error.code}: {error.message}")
    print(f"Problematic query: {query}")

# Close the cursor and connection
cursor.close()
connection.close()

In [ ]:
import pandas as pd
import cx_Oracle

dsn = cx_Oracle.makedsn('10.23.13.62', '1521', service_name='OPERRISKDB')
connection = cx_Oracle.connect('OPERRISK_DS', 'Qwerty2020##', dsn)

drop_query = "DROP TABLE NELLI_CB_NEW_VERSION"

cursor = connection.cursor()
cursor.execute(drop_query)

In [ ]:
import pandas as pd
import cx_Oracle

dsn = cx_Oracle.makedsn('10.23.13.62', '1521', service_name='OPERRISKDB')
connection = cx_Oracle.connect('OPERRISK_DS', 'Qwerty2020##', dsn)

select_query = "SELECT * FROM NELLI_CB_NEW_VERSION"

cursor = connection.cursor()
cursor.execute(select_query)

# Fetch the result of the query
data = cursor.fetchall()

# Get the column names from the cursor description
columns = [col[0] for col in cursor.description]

# Create a DataFrame from the fetched data and column names
df = pd.DataFrame(data, columns=columns)

# Close the cursor and connection
cursor.close()
connection.close()

# Now you have your data in a Pandas DataFrame 'df'